In [ ]:
from my_utils.data import *
from models.network_swinir import *
from models.AdaptNet1015 import AdaptNet, AdaptConfig
import torch
from my_utils.plot import * 
from my_utils.utils import calculate_psnr

In [ ]:
border = 0
window_size = 8
device = 'cuda:1'
ckpt_path = "/home/data1/musong/workspace/python/SwinIR/model_zoo/swinir/005_colorDN_DFWB_s128w8_SwinIR-M_noise50.pth"

model = SwinIR(upscale=1, in_chans=3, img_size=128, window_size=8,
            img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
            mlp_ratio=2, upsampler='', resi_connection='1conv')

pretrained_model = torch.load(ckpt_path)
param_key_g = 'params'
model.load_state_dict(pretrained_model[param_key_g] if param_key_g in pretrained_model.keys() else pretrained_model, strict=True)
model = model.to(device)

conv_cfg = AdaptConfig(mode="conv", num_blocks=3, expansion=2, use_affine=True)
conv_adapt = AdaptNet(c=3, cfg=conv_cfg).to(device)

In [ ]:
img_lq, img_gt = prepare_oom_mr_image()
# img_lq, img_gt = get_image_pair()
img_lq = np.transpose(img_lq if img_lq.shape[2] == 1 else img_lq[:, :, [2, 1, 0]], (2, 0, 1)) 
img_lq = np.clip(img_lq, 0, 1)
img_lq = torch.from_numpy(img_lq).float().unsqueeze(0).to(device) 
img_gt = np.transpose(img_gt if img_gt.shape[2] == 1 else img_gt[:, :, [2, 1, 0]], (2, 0, 1)) 
img_gt = torch.from_numpy(img_gt).float().unsqueeze(0)

with torch.no_grad():
    output = model(img_lq)
    adaptd = conv_adapt(output)

In [ ]:
show_three_images(img_lq, img_gt, adaptd)

In [ ]:
img_gt = (img_gt * 255.0).round().detach().cpu().numpy().astype(np.uint8) 
img_lq = (img_lq * 255.0).round().detach().cpu().numpy().astype(np.uint8) 
output = (output * 255.0).round().detach().cpu().numpy().astype(np.uint8) 
adaptd = (adaptd * 255.0).round().detach().cpu().numpy().astype(np.uint8) 
print(calculate_psnr(img_lq, img_gt, crop_border=0))
print(calculate_psnr(output, img_gt, crop_border=0))
print(calculate_psnr(adaptd, img_gt, crop_border=0))